# 导入模块

残值计算旧版：iPhone 采购价*0.6；非iPhone 采购价*0.4;


残值计算新版：iPhone15 最新采购价*0.4，iPhone16 最新采购价*0.6；非iPhone 采购价*0.425;

毛利的计算逻辑
1、实际毛利计算：
订单状态为：已逾期、已买断、已完成
实际毛利 = 总已收-采购成本

订单状态为：还款中、续租中、未到首期还款日
实际毛利 = 总应收-采购成本



2、应收毛利计算：
应收毛利 = 总应收-采购成本

总已收 = 已收租金+已归还残值/已付买断尾款 + 违约金 +其他账单记账总和（滞纳金或其他扣款）
总应收 = 总租金+预估残值

1）租后表总已收需要查看残值表如果有已处理完成的实际残值，用实际残值，公式如下：
总已收 = 已收租金+已归还残值/已付买断尾款（不为0）
2）租后表总如果没有已处理完成的实际残值，用预估残值，公式如下：
总已收 = 已收租金+预估残值（buyout_balance_payment） 已归还残值/已付买断尾款（为0）

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time , datetime
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [25]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 定义数据库连接

In [26]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 获取租后数据

In [27]:
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,om.order_tag, om.has_actual, tod.sku_attributes, too.money, tos.reality_refund_date_max
-- , cc.name as merchant_name
, om.order_method, cc.channel_type_id, om.order_type, pa.type
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on tprm.order_id = om.id
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join (SELECT t.order_id,sum(case when t.capital_type =6 and t.`status`=2 then t.money end) as other_money 
,sum(case when t.capital_type != 5 AND  t.`status` = 2 then t.money end) as money
from db_digua_business.t_order_other t GROUP BY 1 ) too on too.order_id=om.id
left join (select order_id, max(reality_refund_date) as reality_refund_date_max from db_digua_business.t_order_stages where reality_refund_date is not null GROUP BY order_id) tos on tprm.order_id=tos.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- WHERE tprm.order_create_time >= DATE_ADD(CURRENT_DATE, INTERVAL -12 DAY)
-- and tprm.order_create_time <= DATE_ADD(CURRENT_DATE, INTERVAL -3 DAY)
;
'''

df_zhys = query(sql)
df_zhys.shape

(57008, 70)

In [28]:
sql_rent = '''
select ymos.order_id, ymos.sort, ymos.is_relet, ymos.refund_date  from db_rent.ya_merchant_order_stages ymos
'''
df_rent = query(sql_rent)
df_rent.shape

(2420128, 4)

# 处理数据

In [29]:
df_zhys = df_zhys[df_zhys.type!=4]
df_zhys["activity_source"] = df_zhys["activity_source"].fillna('未知活动') 
df_zhys["channel_name"] = df_zhys["channel_name"].fillna('未知渠道') 

df_zhys.loc[:,"归属渠道"]=df_zhys.apply(lambda x: clean.qudao_type(x["channel_name"],x["activity_source"], x['order_method'], x['channel_type_id'], x['order_type']),axis=1)
df_zhys.loc[:,"租赁方案"]=np.where(df_zhys["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")

# 剔除商家

In [30]:
# 调用Data_Clean类中的drop_merchant函数
df = clean.drop_merchant(df_zhys)
df2 = df.copy()
df_zhys = clean.drop_rejected_merchant(df2)

In [31]:
# sheet = pd.ExcelFile('F:/毛利/2024年二手机回收表.xlsx')
# sheet_names = sheet.sheet_names
# df_sheet = pd.DataFrame()
# for i in range(1, 13):
#     df = sheet.parse(f'{i}月')
#     df.insert(0, '月份', i)
#     df_sheet = pd.concat([df_sheet, df])


In [32]:
# df_sheet_group = df_sheet.groupby('型号').agg({'数量': 'sum', '总回收价格': 'sum', '总采购金额': 'sum', '总官网金额': 'sum'})
# df_sheet_group.loc[:, '总采购价比值'] = (df_sheet_group.总回收价格/df_sheet_group.总采购金额).map(lambda x: format(x, '.2%'))
# df_sheet_group.loc[:, '总官网价比值'] = (df_sheet_group.总回收价格/df_sheet_group.总官网金额).map(lambda x: format(x, '.2%'))
# df_sheet_group[df_sheet_group.index.str.contains('iPhone')]

In [33]:
# df_sheet_group = df_sheet.groupby(['月份', '型号']).agg({'数量': 'sum', '总回收价格': 'sum', '总采购金额': 'sum', '总官网金额': 'sum'})
# df_sheet_group.loc[:, '总采购价比值'] = (df_sheet_group.总回收价格 / df_sheet_group.总采购金额).map(lambda x: format(x, '.2%'))
# df_sheet_group.loc[:, '总官网价比值'] = (df_sheet_group.总回收价格 / df_sheet_group.总官网金额).map(lambda x: format(x, '.2%'))
# df_sheet_group = df_sheet_group.reset_index()
# df_sheet_group[df_sheet_group.型号.str.contains('iPhone')].to_excel('F:/maoli.xlsx', index=False)

# 毛利计算系数

In [34]:

# # 旧版最新系数逻辑
# def ygcz(row):
#     if pd.isna(row['product_name']):
#         return row['采购金额2'] * 0.5
#     elif 'iPhone 16' in row['product_name'].replace(' ', ''):
#         return row['采购金额2'] * 0.6
#     elif 'iPhone15' in row['product_name'].replace(' ', ''):
#         return row['采购金额2'] * 0.4
#     else:
#         return row['采购金额2'] * 0.425

In [35]:
# 奇哥版系数逻辑
def ygcz(product_name, purchase_amount, status, order_finish_date, now_date):
    if 'iPhone' in product_name:
        if status =='已完成':
            if 'iPhone16' in product_name.replace(' ',''):
                return purchase_amount * 0.65
            elif 'iPhone15' in product_name.replace(' ',''):
                if order_finish_date>=pd.to_datetime('2023-10-01') and order_finish_date<pd.to_datetime('2024-10-01'):
                    return purchase_amount * 0.65
                elif order_finish_date>=pd.to_datetime('2024-10-01'):
                    return purchase_amount * 0.55
            elif 'iPhone14' in product_name.replace(' ',''):
                if order_finish_date>=pd.to_datetime('2022-10-01') and order_finish_date<pd.to_datetime('2023-10-01'):
                    return purchase_amount * 0.6
                elif order_finish_date>=pd.to_datetime('2023-10-01') and order_finish_date<pd.to_datetime('2024-10-01'):
                    return purchase_amount * 0.55
                elif order_finish_date>=pd.to_datetime('2024-10-01'):
                    return purchase_amount * 0.5
            else:
                return purchase_amount * 0.5
        else:
            if 'iPhone16' in product_name.replace(' ',''):
                return purchase_amount * 0.65
            elif 'iPhone15' in product_name.replace(' ',''):
                if now_date>=pd.to_datetime('2023-10-01') and now_date<pd.to_datetime('2024-10-01'):
                    return purchase_amount * 0.65
                elif now_date>=pd.to_datetime('2024-10-01'):
                    return purchase_amount * 0.55
            elif 'iPhone14' in product_name.replace(' ',''):
                if now_date>=pd.to_datetime('2022-10-01') and now_date<pd.to_datetime('2023-10-01'):
                    return purchase_amount * 0.6
                elif now_date>=pd.to_datetime('2023-10-01') and now_date<pd.to_datetime('2024-10-01'):
                    return purchase_amount * 0.55
                elif now_date>=pd.to_datetime('2024-10-01'):
                    return purchase_amount * 0.5
            else:
                return purchase_amount * 0.5
    else:
        return purchase_amount * 0.425

# 区分续租和非续租

In [36]:
# 获取订单的最大期数
df_rent.loc[:, 'max_sort'] = df_rent.groupby('order_id')['sort'].transform('max')
# 区分订单是否续租
df_rent_xz = df_rent[df_rent.is_relet>=1]['order_id'].drop_duplicates()
df_rent_fxz = df_rent[df_rent.is_relet==0]#['order_id'].drop_duplicates()
# 取到非续租订单的最大还款日期且日期+1个月和最大期数
df_rent_fxz.loc[:, 'max_refund_date'] = df_rent_fxz.groupby('order_id').refund_date.transform('max')
df_rent_fxz.loc[:, 'max_refund_date'] = df_rent_fxz.max_refund_date + pd.DateOffset(months=1)
df_rent_fxz.loc[:, 'max_sort_fxz'] = df_rent_fxz.groupby('order_id')['sort'].transform('max')
# 提取所需字段
df_rent_fxz = df_rent_fxz[['order_id', 'max_refund_date', 'max_sort', 'max_sort_fxz']].drop_duplicates()

In [37]:
# 表关联
df_zhys_fxz = df_zhys.merge(df_rent_fxz, on='order_id', how='left')
# 判断续租订单并用最大还款日期替换订单完成时间
df_zhys_fxz.loc[:, '订单完成时间'] = np.where(df_zhys_fxz.max_sort_fxz<df_zhys_fxz.max_sort, df_zhys_fxz.max_refund_date, df_zhys_fxz.订单完成时间)
df_zhys_fxz.loc[:, '订单状态'] = np.where(df_zhys_fxz.max_sort_fxz<df_zhys_fxz.max_sort, '已完成', df_zhys_fxz.订单状态)

In [38]:
df_zhys_xz = df_zhys.merge(df_rent_xz, on='order_id', how='right')
df_zhys_xz.loc[:, '租赁方案'] = '租完即送'

In [39]:
# pd.concat([df_zhys_fxz, df_zhys_xz], axis=0)

,id,order_id,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time,update_time,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,订单状态值,订单状态,订单完成时间,true_name,id_card_num,mobile,order_tag,has_actual,sku_attributes,money,reality_refund_date_max,order_method,channel_type_id,order_type,type,归属渠道,租赁方案,max_refund_date,max_sort,max_sort_fxz
0,1.0,8924,A202207050851373,2022年07月,2022-07,2024-10-02,14.0,迪瓜优选,0.0,未知渠道,99新 iPhone 12 Pro 国行 官方质检【租物特惠】,内存：128G 颜色：金色,5150.0,租完归还/可买断,5303.0,4295.40,0.0,2121.20,1.0,357.95,2022-07-05 08:51:37,已买断,12.0,NaT,0.0,4.0,0.0,6416.60,7530.0,7847.20,-3881.52,329.72,8176.92,-1760.32,-3026.92,,1266.60,0.00,3026.92,0.0,11.0,301.0,3551.8,NaN,未知活动,,2024-07-16 11:27:01,2024-07-18 15:50:56,2024-04-17 01:37:33,0.0,0.00,0.0,None,None,0.0,8.0,已完成,2023-07-05 00:00:00,胡水飘,362202199805282550,13027948681,0.0,1.0,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu...",0.00,2023-06-01 09:00:03,0.0,NaN,ZFB_ORDER,NaN,未知渠道,租完归还,2023-07-05,24.0,12.0
1,2.0,9305,A202207131230485,2022年07月,2022-07,2023-07-13,14.0,迪瓜优选,0.0,未知渠道,全新 苹果 iPhone 13 Pro Max 国行 全网通【租物特惠】,内存：256G 颜色：银色 套餐：租完归还-可随时买断,0.0,,9799.0,571.61,0.0,0.00,1.0,571.61,2022-07-13 12:30:48,已完成,1.0,NaT,0.0,0.0,0.0,571.61,12739.0,571.61,0.00,0.00,571.61,0.00,-571.61,,571.61,0.00,571.61,0.0,0.0,0.0,0.0,NaN,未知活动,违约金,2024-07-16 11:27:01,2024-07-18 15:50:56,2022-07-16 10:19:00,0.0,NaN,0.0,None,None,0.0,8.0,已完成,2022-07-16 10:19:00,张素芬,341221199902108727,15585053022,NaN,0.0,"[{""key"":""内存"",""value"":""256G""},{""key"":""颜色"",""valu...",NaN,2022-07-13 15:41:45,0.0,NaN,ZFB_ORDER,NaN,未知渠道,租完归还,2023-07-13,12.0,12.0
2,3.0,9514,A202207151707153,2022年07月,2022-07,2023-07-15,14.0,迪瓜优选,0.0,未知渠道,全新vivo X80 Pro 高通骁龙8蔡司智能5G【租物特惠】,内存：12GB+256G 颜色：假日 属性：租完归还-可随时买断,4794.0,,5999.0,4799.16,0.0,2549.57,2.0,799.86,2022-07-15 17:07:15,已买断,12.0,NaT,0.0,1.0,0.0,7348.73,7798.0,4799.16,-2598.91,2598.91,7398.07,-49.34,-2604.07,,2554.73,0.00,2604.07,0.0,0.0,0.0,0.0,NaN,未知活动,,2024-07-16 11:27:01,2024-07-18 15:50:57,2023-06-30 11:54:29,0.0,NaN,0.0,None,None,0.0,8.0,已完成,2023-06-30 11:54:29,彭友宁,360782200204020216,15297728428,0.0,1.0,"[{""key"":""内存"",""value"":""12GB+256G""},{""key"":""颜色"",...",NaN,2023-05-15 17:40:22,0.0,NaN,ZFB_ORDER,NaN,未知渠道,租完归还,2023-07-15,12.0,12.0
3,4.0,9600,A202207162356303,2022年07月,2022-07,2024-06-16,14.0,迪瓜优选,0.0,未知渠道,全新 苹果 iPhone 13 Pro Max 国行 全网通【租物特惠】,内存：256G 颜色：苍岭绿色 套餐：租完归还-可随时买断,9180.0,,9799.0,6859.20,0.0,3919.60,2.0,1143.20,2022-07-16 23:56:30,已买断,12.0,NaT,0.0,1.0,0.0,10778.80,12739.0,6859.20,-6287.60,6287.60,13146.80,-2368.00,-3966.80,,1598.80,0.00,3966.80,0.0,0.0,0.0,0.0,NaN,未知活动,,2024-07-16 11:27:02,2024-07-18 15:50:57,2022-12-12 13:01:18,0.0,NaN,0.0,None,None,0.0,8.0,已完成,2022-12-13 10:39:58,朱兆成,220181199605113514,17643203514,1.0,1.0,"[{""key"":""内存"",""value"":""256G""},{""key"":""颜色"",""valu...",NaN,2022-12-12 12:57:14,0.0,NaN,ZFB_ORDER,NaN,未知渠道,租完归还,2023-07-16,12.0,12.0
4,5.0,9683,A202207171820512,2022年07月,2022-07,2023-07-17,14.0,迪瓜优选,0.0,未知渠道,vivo Y55s 5G手机 6000mAh巨能量电池【租物特惠】,内存：8GB+128GB 颜色：樱粉流星 属性：租完归还-可随时买断,1214.0,,1699.0,1410.12,0.0,722.07,2.0,235.02,2022-07-17 18:20:51,已买断,12.0,NaT,0.0,27.0,0.0,2132.20,2208.0,1410.12,-797.88,797.88,2208.00,-75.80,-994.00,,918.20,0.00,994.00,0.0,0.0,0.0,0.0,NaN,未知活动,,2024-07-16 11:27:02,2024-07-18 15:50:57,2024-04-20 18:46:54,0.0,0.0

# 毛利计算

In [17]:
def profit_iphone(df_zhys, key):
    df_zhys = df_zhys[(df_zhys['purchase_amount'] > 0) & (df_zhys['订单状态'] != '待收货') & (df_zhys['订单状态'] != '已退款')]
    df_zhys['product_name'] = df_zhys['model_number'] + df_zhys['specification']
    df_zhys['下单日期'] = df_zhys['order_create_time'].dt.date
    df_zhys['下单月份'] = df_zhys['order_create_time'].dt.strftime('%Y-%m')
    df_zhys['下单年份'] = df_zhys['order_create_time'].dt.year
    df_zhys['now_date'] = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))
    df_zhys["内存"] = df_zhys["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
    df_zhys["颜色"] = df_zhys["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
    df_zhys['型号颜色1'] = df_zhys['model_number'].str.cat([df_zhys['内存'], df_zhys['颜色']], sep=' ').str.replace(' ', '')

    df_zhys['型号颜色1'] = df_zhys['型号颜色1'].dropna()
    df_zhys['采购金额2'] = df_zhys['purchase_amount']
    df_zhys = df_zhys[df_zhys['采购金额2'] != 0]
    # df_zhys['预估残值2'] = df_zhys.apply(ygcz, axis=1)
    df_zhys['预估残值2'] = df_zhys.apply(lambda x: ygcz(x['product_name'], x['purchase_amount'], x['订单状态'], x['订单完成时间'], x['now_date']), axis=1)

    df_zhys['总已收2'] = np.where(
        df_zhys['租赁方案'] == '租完即送', df_zhys['rent_received'] +
        np.where(df_zhys['money'].notna(), df_zhys['money'], 0) +
        np.where(df_zhys['订单状态'] == '已完成',
                df_zhys['paid_buyout_balance_payment'], 0) +
        np.where(df_zhys['liquidated_damages'].notna(),
                df_zhys['liquidated_damages'], 0), df_zhys['rent_received'] +
        np.where(df_zhys['money'].notna(), df_zhys['money'], 0) + np.where(
            df_zhys['订单状态'] == '已完成',
            np.where((df_zhys['has_actual'] == 0),
                    np.where(df_zhys['paid_buyout_balance_payment'] == 0,
                            df_zhys['预估残值2'],
                            df_zhys['paid_buyout_balance_payment']),
                    df_zhys['paid_buyout_balance_payment']), 0) +
        np.where(df_zhys['liquidated_damages'].notna(),
                df_zhys['liquidated_damages'], 0))

    df_zhys['total_receivable2'] = np.where(
        df_zhys['租赁方案'] == '租完即送', df_zhys['all_rental'] +
        np.where(df_zhys['money'].notna(), df_zhys['money'], 0),
        df_zhys['all_rental'] + df_zhys['预估残值2'] +
        np.where(df_zhys['money'].notna(), df_zhys['money'], 0))

    status1_list = ['已逾期', '已买断', '已完成']
    status2_list = ['还款中', '续租中', '未到首期还款日']
    df_zhys['实际毛利2'] = np.where(
        df_zhys['rembursement_status'].isin(status2_list),
        df_zhys['total_receivable2'] - df_zhys['purchase_amount'],
        np.where(df_zhys['rembursement_status'].isin(status1_list),
                df_zhys['总已收2'] - df_zhys['purchase_amount'],
                df_zhys['actual_gross_profit']))
    df_zhys['gross_profit_receivable2'] = df_zhys['total_receivable2'] - df_zhys['purchase_amount']

    mon_zhys = df_zhys.groupby(key).agg({
        'order_number': 'size',
        'purchase_amount': 'sum',
        'all_deposit': 'sum',
        'all_rental': 'sum',
        'buyout_balance_payment': 'sum',
        'total_receivable': 'sum',
        'total_receivable2': 'sum',
        'rent_received': 'sum',
        'unearned_rent_revenue': 'sum',
        'paid_buyout_balance_payment': 'sum',
        'total_received': 'sum',
        'total_unreceived': 'sum',
        'unrecovered_cost': 'sum',
        'overdue_total_unreceived': 'sum',
        'overdue_loss': 'sum',
        'gross_profit_receivable': 'sum',
        'gross_profit_receivable2': 'sum',
        'actual_gross_profit': 'sum',
        '采购金额2': 'sum',
        '预估残值2': 'sum',
        '总已收2': 'sum',
        '实际毛利2': 'sum'
    })
    mon_zhys['overdue_rate'] = mon_zhys.overdue_total_unreceived / mon_zhys.total_receivable2
    mon_zhys['loss_rate'] = mon_zhys.overdue_loss / mon_zhys.purchase_amount
    mon_zhys['receivable_profite_rate'] = mon_zhys.gross_profit_receivable2 / mon_zhys.purchase_amount  # 应收毛利率(应收毛利/采购成本)
    mon_zhys['profite_rate'] = mon_zhys.实际毛利2 / mon_zhys.purchase_amount  # 实际毛利率 (实际毛利/采购成本)

    mon_zhys = mon_zhys[[
        'order_number', 'purchase_amount', '采购金额2', 'all_deposit',
        'all_rental', 'buyout_balance_payment', '预估残值2', 'total_receivable2',
        'total_receivable', 'rent_received', 'unearned_rent_revenue',
        'paid_buyout_balance_payment', '总已收2', 'total_received',
        'total_unreceived', 'unrecovered_cost', 'overdue_total_unreceived',
        'overdue_rate', 'overdue_loss', 'loss_rate', 'gross_profit_receivable',
        'gross_profit_receivable2', 'receivable_profite_rate',
        'actual_gross_profit', '实际毛利2', 'profite_rate'
    ]]
    mon_zhys2 = mon_zhys.reset_index()

    mon_zhys2 = mon_zhys2.rename(
        columns={  #'search_time':'下单月份'
            'order_number': '订单数',
            'purchase_amount': '采购金额',
            'all_deposit': '总押金',
            'all_rental': '总租金',
            'buyout_balance_payment': '预估残值',
            'total_receivable': '总应收',
            'total_receivable2': '总应收2',
            'rent_received': '已收租金',
            'unearned_rent_revenue': '未收租金',
            'paid_buyout_balance_payment': '已归还残值/已付买断尾款',
            'total_received': '总已收',
            'total_unreceived': '总未收',
            'unrecovered_cost': '未回收成本',
            'overdue_total_unreceived': '逾期总未收',
            'overdue_rate': '应收逾期率',
            'overdue_loss': '逾期损失',
            'loss_rate': '成本逾期率',
            'gross_profit_receivable': '应收毛利',
            'gross_profit_receivable2': '应收毛利2',
            'receivable_profite_rate': '应收毛利率(应收毛利/采购成本)',
            'actual_gross_profit': '实际毛利',
            'profite_rate': '实际毛利率 (实际毛利/采购成本)'
        })
    mon_zhys2['预计坏账(逾期总未收/采购金额)'] = mon_zhys2['逾期总未收'] / mon_zhys2['采购金额']
    mon_zhys2['实际毛利(应收毛利-预计坏账)'] = mon_zhys2['应收毛利率(应收毛利/采购成本)'] - mon_zhys2[
        '预计坏账(逾期总未收/采购金额)']
    df_zhys['订单状态'].drop_duplicates()
    # mon_zhys2['预估残值2'] = mon_zhys2['预估残值2'].apply(lambda x: format(x, '.2%'))
    # mon_zhys2['应收毛利2'] = mon_zhys2['应收毛利2'].apply(lambda x: format(x, '.2%'))
    # mon_zhys2['实际毛利2'] = mon_zhys2['实际毛利2'].apply(lambda x: format(x, '.2%'))
    return mon_zhys2, df_zhys

In [18]:
# df_profit_all, df_zhys  = profit_iphone(df_zhys, key='下单月份')
# df_profit_all.to_excel('F:/需求/总需求/毛利2.xlsx', index=False)
# df_profit_all1, df_zhys1  = profit_iphone(df_zhys_xz)

In [19]:
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')

In [20]:
# df_profit_all_new = df_profit_all[['下单月份','订单数','采购金额','总押金','总租金','预估残值','预估残值2','总应收','总应收2','已收租金','未收租金','已归还残值/已付买断尾款','总已收','总已收2','总未收','未回收成本','逾期总未收','逾期损失','应收毛利','应收毛利2','实际毛利','实际毛利2','应收毛利率(应收毛利/采购成本)','实际毛利率 (实际毛利/采购成本)','应收逾期率','成本逾期率','预计坏账(逾期总未收/采购金额)','实际毛利(应收毛利-预计坏账)']]#.to_excel('F:/月份毛利.xlsx')
# with pd.ExcelWriter(f'F:/毛利/租后数据_202502V2(1)_{t_date}.xlsx', engine='openpyxl') as writer:
#     df_profit_all_new.to_excel(writer, sheet_name='20250221', index=False)

In [21]:
## 资方
sql_zf = '''
SELECT tprm.order_number, om.merchant_name,tod.sku_attributes, tso.price as 碎屏险价格, om.create_time create_time_zf,
tod.new_actual_money '应收金额', tprm.purchase_amount '采购成本', (tprm.rent_received) '已收租金', tprm.overdue_type, top.residue_money '押金'
,case when ym.`name`='维客自营账户' then '维客自营账户' 
			when ym.`name`='海南鸿丰商业保理有限公司' then '海南鸿丰商业保理有限公司'
			when ym.`name`='成都越蓉金科科技有限公司' then '成都越蓉金科科技有限公司'
			when ym.`name`='杭州金锤网络信息服务有限公司' then '杭州金锤网络信息服务有限公司'
			else '其他' end '资方'
FROM db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join (SELECT order_id, residue_money from db_digua_business.t_order_pledge where residue_money>=0) top on top.order_id=om.id
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id
-- where om.`status`=4
'''
df_zf = query(sql_zf)

In [22]:
df_zf_merge = df_zf[['order_number', 'create_time_zf', '资方']].merge(df_zhys, on='order_number', how='left')
df_zf_merge.loc[:, 'year'] = df_zf_merge.create_time_zf.dt.year

mon_zhys2, df_profit_zf  = profit_iphone(df_zf_merge, ['year', '资方'])
# df_profit_zf[['下单日期', 'total_receivable2', 'purchase_amount', 'rembursement_status']]
mon_zhys2.sort_values('资方', ascending=False).to_excel('F:/需求/总需求/资方毛利41.xlsx', index=False)

In [ ]:
# df_zf_merge_n = df_zf_merge[(df_zf_merge.year==2025)&(df_zf_merge.资方=='维客自营账户')]
# mon_zhys2, df_profit_zf  = profit_iphone(df_zf_merge_n, ['year', '资方'])
# df_profit_zf_2 = df_profit_zf[['order_number', 'total_receivable2', '总已收2', 'purchase_amount', 'rembursement_status', '实际毛利2']].groupby('rembursement_status').agg({'total_receivable2': 'sum', '总已收2': 'sum', 'purchase_amount': 'sum', '实际毛利2': 'sum'})
# # df_profit_zf_2.loc[:, 'rate'] = df_profit_zf_2.total_receivable2-df_profit_zf_2.purchase_amount
# # df_profit_zf_2[df_profit_zf_2.rate>df_profit_zf_2.purchase_amount]
# df_profit_zf_2